In [3]:
from module.bis_selenium import BisSelenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import polars as pl
from bs4 import BeautifulSoup
import lxml

In [2]:
import time
import os
from dotenv import load_dotenv; load_dotenv()

driver_path = r'C:\Users\341137\src\driver\chromedriver.exe'
auth_id = os.environ.get('AUTH_ID')
auth_pass = os.environ.get('AUTH_PASS')

bis_selenium = BisSelenium(driver_path)
bis_selenium.login(auth_id, auth_pass)
bis_selenium.move_to_timesales()

# element_selecter
pattern_selecter = bis_selenium.find_element(By.CSS_SELECTOR, '#displayPatternSelect1')
date_selecter = bis_selenium.find_element(By.CSS_SELECTOR, '#date')
hour_selecter = bis_selenium.find_element(By.CSS_SELECTOR, '#hour')  
report_viewer = bis_selenium.find_element(By.CSS_SELECTOR, '#viewJikanichiran')

# 条件選択部分
bis_selenium.find_elements(By.CSS_SELECTOR, '#kaishaCodeRadio')[2].click()
Select(pattern_selecter).select_by_value('7')

# 組織選択
org_list = ['0003_12000', '0004_13000']
bis_selenium.select_orgs(*org_list)

# 商品分類選択
product_list = ['[211]グロサリー', '[214]リカー', '[221]デイリーフーズ']
bis_selenium.select_products(*product_list)

# 日付選択
bis_selenium.input_without_JS(date_selecter, '2023/07/30')
Select(hour_selecter).select_by_value('10')

# 画面遷移
report_viewer.click()
bis_selenium.move_to_new_window()
first_frame = bis_selenium.find_element(By.CSS_SELECTOR, '#Dottom')
bis_selenium.switch_to.frame(first_frame)

# first_fetch
time_selector = bis_selenium.find_element(By.CSS_SELECTOR, '#TIME_SELECT')
bottom_frame = bis_selenium.find_element(By.CSS_SELECTOR, '#bottomFrame')
bis_selenium.switch_to.frame(bottom_frame)
html = bis_selenium.page_source.encode('utf-8')

bis_selenium.quit()

0003_12000
0004_13000
switch_window:時間帯別売上速報(一覧)


In [ ]:
soup = BeautifulSoup(html, 'lxml')
table = soup.find('tbody')
header_base = table.find('tr').find_all('td')
headers = [header.get_text().replace('\n', '').replace(' ', '') for header in header_base]
rows_base = table.find_all('tr')[1:]
body = []
for row_base in rows_base:
  row = []
  for row_elem in row_base.find_all('td'):
    row.append(row_elem.get_text().replace('\n', '').replace(' ', '') )
  body.append(row)
  
df = pl.DataFrame(data=body, schema=headers)
selected_cols = [col for col in df.columns if '売上高' in col and '前日' not in col]
df_selected = df.select(selected_cols)
for col in df_selected.columns:
  df_selected = (df_selected
                 .with_columns(
                   pl.col(col)
                   .str.replace(" ", "0")
                   .str.replace("\xa0", "0")
                   .str.replace(",", "")
                   .cast(pl.Float64)
                   )
                 )
base_cols = ['店舗CD', '店舗', 'グループCD', 'グループ']
df_base = df.select(base_cols)
df_concat = pl.concat([df_base, df_selected], how='horizontal')

In [123]:
for i in range(2):
  df_concat.write_csv(f'{str(i)}.csv')